In [1]:
using LinearAlgebra, Test

# params
t_2 = 1.0
θ_1 = 1.0
θ_2 = 1.0
θ(t) = t < t_2 ? θ_1 : θ_2
t = 0.0
N = 5

5

In [2]:
# intermediate vals
N_total = 2*N;
setsize = (N+1)^2;

In [3]:
function get_row_block(d_i)    
    # note that all rows sum up to zero by construction
    dl_block = -(z->((N_total-z)/N_total)*θ(t)).(d_i:(d_i+N))
    d_block = -dl_block/2
    du_block = -dl_block/2
    if (d_i == N) # there's no diffusion to 2nd cohort in the last block
        dl_block = dl_block/2
    end
    
    mat = diagm(0 => dl_block, 1 => d_block, (1+N)=>du_block) 
    
    # modify the last row, where first cohort products are fully recognized
    mat[(N+1),(N+2)] = 0.0
    mat[(N+1),end] *= 2 
    mat[(N+1),:] /= 2 # if departure rate is halved for full recognized state
    
    # fill the rest in row block with zeros
    if (d_i == 0)
        return [mat[1:(N+1),:] zeros(N+1, (N+1)*(N-1))]
    elseif (d_i >= (N-1))
        return [zeros(N+1, d_i*(N+1)) mat[1:(N+1),:]][:,1:setsize]
    else
        return [zeros(N+1, d_i*(N+1)) mat[1:(N+1),:] zeros(N+1, (N-1-d_i)*(N+1))]
    end
end

transition_mat = vcat(get_row_block.(0:N)...)

@test all(transition_mat*ones(setsize,1) .== 0.0) # all row sum should be zero

Test Passed